<a href="https://colab.research.google.com/github/MaryamNourii/ChatBot/blob/main/Basic_Fa_GPT2_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.2 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForCausalLM

model_name = "HooshvareLab/gpt2-fa"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForCausalLM.from_pretrained(model_name)

def generate_response(input_text, max_length=500):
    encoded_input = tokenizer(input_text, return_tensors='tf')

    generated_output = model.generate(
        input_ids=encoded_input['input_ids'],
        attention_mask=encoded_input['attention_mask'],
        max_length=max_length,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True
    )

    generated_text = tokenizer.decode(generated_output[0], skip_special_tokens=True)

    return generated_text.strip()



In [ ]:
# input_text = 'سلام چطوری؟ من نمی دونم چیکار کنم با این درسا، لطفا زاهنماییم کن'
input_text = 'سلام چطوری؟ من چیکار کنم؟'

resp = generate_response(input_text)

In [ ]:
print("Generated response: {}".format(resp))

Generated response: سلام چطوری؟ من نمی دونم چیکار کنم با این درسا، لطفا زاهنماییم کنارتون رو عوض کنید. خب حالا بریم سراغ اصل مطلب. اول از همه باید بگم که من به عنوان یک برنامه نویس حرفه‌ای، همیشه سعی می‌کنم خودم رو به روز نگه دارم و همیشه در حال یادگیری باشم. به همین دلیل هم هست که خیلی از برنامه نویسان تازه کار، به دنبال یادگیری مهارت‌های برنامه نویسی هستند. در این مقاله قصد دارم به شما یاد بدم که چطور با استفاده از زبان‌ها و فریم ورک هایی مثل React، Angular، Vue، React و … به راحتی و با صرف کمترین زمان، برنامه‌ی خود را بسازید. پس با من همراه باشید. برای شروع، شما نیاز دارید که یک وب سایت یا یک اپلیکیشن را ایجاد کنید و سپس آن را در مرورگر خود باز کرده و شروع به ساخت یک وبسایت یا اپلیکیشن جدید بکنید. این کار بسیار ساده است، فقط کافی است که به سایت خود بروید و در آن ثبت نام کنید، سپس یک حساب کاربری بسازید و وارد آن شوید. بعد از ساخت وبسایت، باید به سراغ ساخت اپلیکیشن بروید. شما باید یک اکانت بسازید تا بتوانید از طریق آن، اپلیکیشن یا وب سایتی که ساخته‌اید را به صورت رایگان در اختیار دیگر

In [35]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, TFGPT2LMHeadModel

model_name = "HooshvareLab/gpt2-fa"
tokenizer = AutoTokenizer.from_pretrained(model_name)
gpt2_model = TFGPT2LMHeadModel.from_pretrained(model_name)

df = pd.read_csv('percQA_CleanData.csv', encoding='utf-8')

questions = df['question'].tolist()
answers = df['answer'].tolist()

max_input_length = 512
max_target_length = 128

tokenized_questions = [tokenizer.encode(q, add_special_tokens=True) for q in questions]
tokenized_answers = [tokenizer.encode(a, add_special_tokens=True) for a in answers]

encoder_input_ids = tf.keras.preprocessing.sequence.pad_sequences(tokenized_questions, maxlen=max_input_length, dtype='int32', padding='post', truncating='post')
encoder_attention_mask = tf.cast(tf.math.not_equal(encoder_input_ids, 0), dtype=tf.int32)
decoder_input_ids = tf.keras.preprocessing.sequence.pad_sequences(tokenized_answers, maxlen=max_target_length, dtype='int32', padding='post', truncating='post')[:, :-1]
decoder_target_ids = tf.keras.preprocessing.sequence.pad_sequences(tokenized_answers, maxlen=max_target_length, dtype='int32', padding='post', truncating='post')[:, 1:]
decoder_attention_mask = tf.cast(tf.math.not_equal(decoder_input_ids, 0), dtype=tf.int32)

class ConversationDataset(tf.keras.utils.Sequence):
    def __init__(self, encoder_input_ids, encoder_attention_mask, decoder_input_ids, decoder_attention_mask, decoder_target_ids, batch_size):
        self.encoder_input_ids = encoder_input_ids
        self.encoder_attention_mask = encoder_attention_mask
        self.decoder_input_ids = decoder_input_ids
        self.decoder_attention_mask = decoder_attention_mask
        self.decoder_target_ids = decoder_target_ids
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.encoder_input_ids) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_encoder_input_ids = self.encoder_input_ids[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_encoder_attention_mask = self.encoder_attention_mask[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_decoder_input_ids = self.decoder_input_ids[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_decoder_attention_mask = self.decoder_attention_mask[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_decoder_target_ids = self.decoder_target_ids[idx * self.batch_size:(idx + 1) * self.batch_size]
        return {'encoder_inputs': batch_encoder_input_ids, 'encoder_attention_mask': batch_encoder_attention_mask, 'decoder_inputs': batch_decoder_input_ids, 'decoder_attention_mask': batch_decoder_attention_mask}, batch_decoder_target_ids

batch_size = 2
dataset = ConversationDataset(encoder_input_ids, encoder_attention_mask, decoder_input_ids, decoder_attention_mask, decoder_target_ids, batch_size)

encoder_inputs = tf.keras.layers.Input(shape=(max_input_length,), dtype=tf.int32, name='encoder_inputs')
encoder_attention_mask = tf.keras.layers.Input(shape=(max_input_length,), dtype=tf.int32, name='encoder_attention_mask')
encoder_output = gpt2_model({'input_ids': encoder_inputs, 'attention_mask': encoder_attention_mask})[0]
decoder_inputs = tf.keras.layers.Input(shape=(max_target_length,), dtype=tf.int32, name='decoder_inputs')
decoder_attention_mask = tf.keras.layers.Input(shape=(max_target_length,), dtype=tf.int32, name='decoder_attention_mask')
decoder_input_embeddings = gpt2_model({'input_ids': decoder_inputs, 'attention_mask': decoder_attention_mask})[0]
decoder_input_embeddings = tf.keras.layers.Dropout(0.1)(decoder_input_embeddings)
decoder_output = tf.keras.layers.Dense(gpt2_model.config.vocab_size, activation='softmax')(decoder_input_embeddings)

model = tf.keras.models.Model(inputs=[encoder_inputs, encoder_attention_mask, decoder_inputs, decoder_attention_mask], outputs=decoder_output)

model.summary()

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy')

epochs = 5
model.fit(dataset, epochs=epochs)


questions = 'سلام '
max_output_length = 128

tokenized_questions = [tokenizer.encode(q, add_special_tokens=True) for q in questions]
encoder_input_ids = tf.keras.preprocessing.sequence.pad_sequences(tokenized_questions, maxlen=max_input_length, dtype='int32', padding='post', truncating='post')
encoder_attention_mask = tf.cast(tf.math.not_equal(encoder_input_ids, 0), dtype=tf.int32)
decoder_input_ids = np.zeros((len(questions), max_output_length), dtype='int32')
decoder_input_ids[:, 0] = tokenizer.cls_token_id

for i in range(1, max_output_length):
    decoder_attention_mask = tf.cast(tf.math.not_equal(decoder_input_ids, 0), dtype=tf.int32)
    predictions = model([encoder_input_ids, encoder_attention_mask, decoder_input_ids, decoder_attention_mask])
    predictions = predictions[:, i-1, :]
    predicted_id = tf.argmax(predictions, axis=-1).numpy()
    decoder_input_ids[:, i] = predicted_id
    if np.all(predicted_id == tokenizer.sep_token_id):
        break

responses = [tokenizer.decode(output, skip_special_tokens=True) for output in decoder_input_ids]
responses

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at HooshvareLab/gpt2-fa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


ResourceExhaustedError: ignored